In [ ]:
!pip install opencv-python

In [ ]:
!pip install --upgrade pyqt5 lxml

In [1]:
import os
import wget

In [2]:
customModelName = 'card_localization'
pretrainedModelName = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
pretrainedModelUrl = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
tfRecordScriptName = 'generate_tfrecord.py'
labelMapName = 'label_map.pbtxt'

paths = {
    'workspacePath': os.path.join('tf_localization', 'workspace'),
    'scriptsPath': os.path.join('tf_localization', 'scripts'),
    'apiModelPath': os.path.join('tf_localization', 'models'),
    'annotationPath': os.path.join('tf_localization', 'workspace', 'annotations'),
    'imagePath': os.path.join('tf_localization', 'workspace', 'images'),
    'modelPath': os.path.join('tf_localization', 'workspace', 'models'),
    'pretrainedModelPath': os.path.join('tf_localization', 'workspace', 'pre-trained-models'),
    'checkpointPath': os.path.join('tf_localization', 'workspace', 'models', customModelName),
    'outputPath': os.path.join('tf_localization', 'workspace', 'models', customModelName, 'export'),
    'tfLitePath': os.path.join('tf_localization', 'workspace', 'models', customModelName, 'tfliteexport'),
    'protocPath': os.path.join('tf_localization', 'protoc')
}

files = {
    'pipelineConfig': os.path.join('tf_localization', 'workspace', 'models', customModelName, 'pipeline.config'),
    'tfRecordScript': os.path.join(paths['scriptsPath'], tfRecordScriptName),
    'labelMap': os.path.join(paths['annotationPath'], labelMapName)
}

In [3]:
for path in paths.values():
    if not os.path.exists(path):
        os.makedirs(path)

In [5]:
objectDetectionReposPath = os.path.join(paths['apiModelPath'], 'research', 'object_detection')

if not os.path.exists(objectDetectionReposPath):
    !git clone https://github.com/tensorflow/models {paths['apiModelPath']}

In [ ]:
url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
wget.download(url)
!move protoc-3.15.6-win64.zip {paths['protocPath']}
!cd {paths['protocPath']} && tar -xf protoc-3.15.6-win64.zip
#os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['protocPath'], 'bin'))   
!cd tf_localization/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
!cd tf_localization/models/research/slim && pip install -e .

In [ ]:
!pip install wrapt --upgrade

In [21]:
verificationScriptPath = os.path.join(paths['apiModelPath'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
!python {verificationScriptPath}

2022-09-13 14:19:29.385457: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'nvcuda.dll'; dlerror: nvcuda.dll not found
2022-09-13 14:19:29.385523: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-13 14:19:29.393352: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: DESKTOP-M46MH9P
2022-09-13 14:19:29.393464: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: DESKTOP-M46MH9P
Running tests under Python 3.9.13: D:\conda_envs\jugio\python.exe
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
2022-09-13 14:19:29.470449: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX AVX2
To enable them in other operations, rebuild TensorFlow with the

In [7]:
import object_detection

In [ ]:
wget.download(pretrainedModelUrl)
!move {pretrainedModelName+'.tar.gz'} {paths['pretrainedModelPath']}
!cd {paths['pretrainedModelPath']} && tar -zxvf {pretrainedModelName+'.tar.gz'}

In [8]:
labels = [{'name':'card', 'id':1}]

with open(files['labelMap'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

In [9]:
if not os.path.exists(files['tfRecordScript']):
    !git clone https://github.com/nicknochnack/GenerateTFRecord {paths['scriptsPath']}

In [10]:
!python {files['tfRecordScript']} -x {os.path.join(paths['imagePath'], 'train')} -l {files['labelMap']} -o {os.path.join(paths['annotationPath'], 'train.record')} 
!python {files['tfRecordScript']} -x {os.path.join(paths['imagePath'], 'test')} -l {files['labelMap']} -o {os.path.join(paths['annotationPath'], 'test.record')} 

Successfully created the TFRecord file: tf_localization\workspace\annotations\train.record
Successfully created the TFRecord file: tf_localization\workspace\annotations\test.record


In [11]:
!copy {os.path.join(paths['pretrainedModelPath'], pretrainedModelName, 'pipeline.config')} {os.path.join(paths['checkpointPath'])}

        1 file(s) copied.


In [12]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [13]:
config = config_util.get_configs_from_pipeline_file(files['pipelineConfig'])

In [14]:
config

{'model': ssd {
   num_classes: 90
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.009999999776482582
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.996999979019165
         scale: true
         epsilon: 0.0010000000474974513
       }
     }
     use_depthwise: true
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   matc

In [15]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['pipelineConfig'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config) 

In [16]:
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['pretrainedModelPath'], pretrainedModelName, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['labelMap']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['annotationPath'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['labelMap']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['annotationPath'], 'test.record')]

In [17]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(files['pipelineConfig'], "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text) 

In [18]:
trainingScript = os.path.join(paths['apiModelPath'], 'research', 'object_detection', 'model_main_tf2.py')

In [19]:
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=2000".format(trainingScript, paths['checkpointPath'],files['pipelineConfig'])

In [20]:
print(command)

python tf_localization\models\research\object_detection\model_main_tf2.py --model_dir=tf_localization\workspace\models\card_localization --pipeline_config_path=tf_localization\workspace\models\card_localization\pipeline.config --num_train_steps=2000


In [24]:
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(trainingScript, paths['checkpointPath'],files['pipelineConfig'], paths['checkpointPath'])

In [25]:
print(command)

python tf_localization\models\research\object_detection\model_main_tf2.py --model_dir=tf_localization\workspace\models\card_localization --pipeline_config_path=tf_localization\workspace\models\card_localization\pipeline.config --checkpoint_dir=tf_localization\workspace\models\card_localization


In [37]:
exportScript = os.path.join(paths['apiModelPath'], 'research', 'object_detection', 'exporter_main_v2.py ')

In [38]:
command = "python {} --input_type=image_tensor --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={}".format(exportScript ,files['pipelineConfig'], paths['checkpointPath'], paths['outputPath'])

In [39]:
print(command)

python tf_localization\models\research\object_detection\exporter_main_v2.py  --input_type=image_tensor --pipeline_config_path=tf_localization\workspace\models\card_localization\pipeline.config --trained_checkpoint_dir=tf_localization\workspace\models\card_localization --output_directory=tf_localization\workspace\models\card_localization\export


In [40]:
tfLiteScript = os.path.join(paths['apiModelPath'], 'research', 'object_detection', 'export_tflite_graph_tf2.py ')

In [41]:
command = "python {} --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={}".format(tfLiteScript ,files['pipelineConfig'], paths['checkpointPath'], paths['tfLitePath'])

In [42]:
print(command)

python tf_localization\models\research\object_detection\export_tflite_graph_tf2.py  --pipeline_config_path=tf_localization\workspace\models\card_localization\pipeline.config --trained_checkpoint_dir=tf_localization\workspace\models\card_localization --output_directory=tf_localization\workspace\models\card_localization\tfliteexport


In [3]:
tfLiteFolderPath = os.path.join(paths['tfLitePath'], 'saved_model')
tfLiteModelPath = os.path.join(paths['tfLitePath'], 'saved_model', 'detect.tflite')

In [4]:
command = "tflite_convert \
--saved_model_dir={} \
--output_file={} \
--input_shapes=1,300,300,3 \
--input_arrays=normalized_input_image_tensor \
--output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' \
--inference_type=FLOAT \
--allow_custom_ops".format(tfLiteFolderPath, tfLiteModelPath, )

In [5]:
print(command)

tflite_convert --saved_model_dir=tf_localization\workspace\models\card_localization\tfliteexport\saved_model --output_file=tf_localization\workspace\models\card_localization\tfliteexport\saved_model\detect.tflite --input_shapes=1,300,300,3 --input_arrays=normalized_input_image_tensor --output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' --inference_type=FLOAT --allow_custom_ops


In [3]:
!pip install tflite-support

     -------------------------------------- 454.0/454.0 kB 1.3 MB/s eta 0:00:00
     -------------------------------------- 213.3/213.3 kB 1.5 MB/s eta 0:00:00
     -------------------------------------- 195.9/195.9 kB 1.3 MB/s eta 0:00:00


In [4]:
from tflite_support.metadata_writers import object_detector
from tflite_support.metadata_writers import writer_utils

In [5]:
ObjectDetectorWriter = object_detector.MetadataWriter
_modelPath = os.path.join(paths['tfLitePath'], 'saved_model', 'detect.tflite')

_labelFile = os.path.join(paths['tfLitePath'], 'saved_model', 'labels.txt')
_saveToPath = "detect_v2.tflite"

_inputNormMean = 127.5
_inputNormStd = 127.5

writer = ObjectDetectorWriter.create_for_inference(
    writer_utils.load_file(_modelPath), [_inputNormMean], [_inputNormStd],
    [_labelFile])

print(writer.get_metadata_json())

writer_utils.save_file(writer.populate(), _saveToPath)

{
  "name": "ObjectDetector",
  "description": "Identify which of a known set of objects might be present and provide information about their positions within the given image or a video stream.",
  "subgraph_metadata": [
    {
      "input_tensor_metadata": [
        {
          "name": "image",
          "description": "Input image to be detected.",
          "content": {
            "content_properties_type": "ImageProperties",
            "content_properties": {
              "color_space": "RGB"
            }
          },
          "process_units": [
            {
              "options_type": "NormalizationOptions",
              "options": {
                "mean": [
                  127.5
                ],
                "std": [
                  127.5
                ]
              }
            }
          ],
          "stats": {
            "max": [
              1.0
            ],
            "min": [
              -1.0
            ]
          }
        }
      ],
     